In [11]:
import backtrader as bt
import setup_path
import pandas as pd
from datetime import datetime, timedelta
from alanq.data import StockDataManager
from alanq.backtest import StockPickerWorker
from alanq.factors import RegressAnglePicker
import math

class TurtleLongWithSelection(bt.Strategy):
    """
    海龜交易法 - 只做多版本 (Backtrader)
    
    規則摘要：
    - 入場：
        系統1：收盤價突破過去 20 日最高價 → 建多單
        系統2：收盤價突破過去 55 日最高價 → 建多單
        (任一系統給出多頭訊號即可，只要目前沒有持倉)
    - 出場：
        系統1：跌破過去 10 日最低價 → 全部出場
        系統2：跌破過去 20 日最低價 → 全部出場
        以及：跌破「入場價 - 2N」的停損價 → 全部出場
    - N 值：ATR(20)
    - 部位大小：單筆風險 = 帳戶淨值 * risk_per_trade
                  停損距離 = 2N
    - 加碼：每漲 0.5N 加碼 1 單位，最多 max_units 單位
    - 僅做多，不做空
    """

    params = dict(
        # 趨勢參數
        sys1_entry=20,   # 系統1突破 (高點)
        sys1_exit=10,    # 系統1出場 (低點)
        sys2_entry=55,   # 系統2突破 (高點)
        sys2_exit=20,    # 系統2出場 (低點)

        # 波動與風險控管
        atr_period=20,
        risk_per_trade=0.01,   # 單筆風險佔帳戶淨值比例 (1%)
        atr_mult_stop=2.0,     # 停損距離 = 2N
        atr_step_pyramid=0.5,  # 每 0.5N 加碼一次
        max_units=4,           # 單一商品最多單位

        point_value=1.0,       # 每一價格單位的價值 (股票通常 = 1；期貨要自己設)
        
        # 選股相關參數
        selection_data_manager=None,  # 用於選股的資料管理器（包含一個月前的資料）
        selection_lookback_days=30,    # 選股回測時間（天數）
        rebalance_frequency=1,        # 重新選股的頻率（每 N 個交易日）
    )

    def __init__(self):
        # 為每個資料源初始化技術指標
        self.indicators = {}
        self.orders = {}
        self.units = {}
        self.entry_prices = {}
        self.stop_prices = {}
        self.next_add_prices = {}
        
        # 為每個資料源創建指標
        for i, data in enumerate(self.datas):
            symbol = data._name if hasattr(data, '_name') else f'data{i}'
            self.indicators[symbol] = {
                'atr': bt.ind.ATR(data, period=self.p.atr_period),
                's1_high': bt.ind.Highest(data.high(-1), period=self.p.sys1_entry),
                's1_low_exit': bt.ind.Lowest(data.low(-1), period=self.p.sys1_exit),
                's2_high': bt.ind.Highest(data.high(-1), period=self.p.sys2_entry),
                's2_low_exit': bt.ind.Lowest(data.low(-1), period=self.p.sys2_exit),
            }
            self.orders[symbol] = None
            self.units[symbol] = 0
            self.entry_prices[symbol] = None
            self.stop_prices[symbol] = None
            self.next_add_prices[symbol] = None
        
        # 選股相關
        self.selected_symbols = set()  # 當前選中的股票
        self.last_selection_date = None
        self.trading_day_count = 0
        
        # 選股配置：角度大於5度小於20度
        self.stock_picker_config = [{
            'class': RegressAnglePicker,
            'threshold_ang_min': 5.0,
            'threshold_ang_max': 20.0,
            'reversed': False
        }]

    # 計算每個 unit 的張數/股數
    def _calc_unit_size(self, symbol):
        """計算部位大小，嚴格防止 NaN 擴散"""
        
        # 1. 取得帳戶價值
        value = self.broker.getvalue()
        # 防呆：如果帳戶已經壞掉了，就回傳 0
        if value is None or math.isnan(value) or value <= 0:
            return 0
        
        risk_amount = value * self.p.risk_per_trade

        # 2. 取得 ATR (N值)
        N = self.indicators[symbol]['atr'][0]
        
        # 防呆：ATR 必須是有效的正數
        # math.isnan(N) 是最關鍵的檢查
        if N is None or math.isnan(N) or N <= 0:
            return 0

        # 3. 計算單位風險
        unit_risk = self.p.atr_mult_stop * N * self.p.point_value
        
        # 防呆：除數不能為 0 或 NaN
        if math.isnan(unit_risk) or unit_risk <= 0:
            return 0

        # 4. 計算數量
        size_float = risk_amount / unit_risk
        
        # 最後防呆：確保算出來的 size 是有效數字
        if math.isnan(size_float) or math.isinf(size_float) or size_float <= 0:
            return 0
        
        try:
            size = int(size_float)
            return max(size, 0)
        except:
            return 0
    
    def _perform_stock_selection(self, current_date):
        """
        執行選股掃描 (改良版：包含代碼對齊與除錯資訊)
        """
        if self.p.selection_data_manager is None:
            self.log("警告: 未設定 selection_data_manager，無法執行選股")
            return
        
        # 1. 計算日期範圍
        try:
            # 結束日：昨天的日期 (避免用到當日未來資料)
            selection_end_date = (current_date - timedelta(days=1)).strftime('%Y-%m-%d')
            # 開始日：回推 N 天
            selection_start_date = (current_date - timedelta(days=self.p.selection_lookback_days)).strftime('%Y-%m-%d')
        except Exception as e:
            self.log(f"日期計算錯誤: {e}")
            return

        try:
            # 2. 取得資料子集 (Subset)
            # 注意：若資料量極大，這一步會較慢
            selection_data = self.p.selection_data_manager.subset(selection_start_date, selection_end_date)
            
            # 3. 執行選股 Worker
            picker_worker = StockPickerWorker(selection_data, self.stock_picker_config)
            picker_worker.fit()
            
            # 取得原始選股結果 (可能是 int, 或含 .TW 的字串)
            raw_picker_symbols = picker_worker.choice_symbols
            
            # =========================================================
            # 4. 關鍵改良：代碼對齊 (Symbol Alignment)
            # =========================================================
            
            # 取得 Backtrader 中目前所有已載入的 Data Feed 名稱
            # 建立一個 mapping 以便快速查找: {'2330': '2330', '2330.TW': '2330.TW'}
            bt_data_names = {d._name: d._name for d in self.datas}
            
            final_matched_symbols = set()
            
            for raw_s in raw_picker_symbols:
                s_str = str(raw_s).strip() # 強制轉字串並去空白
                
                # 情況 A: 完全匹配 (例如 Picker="2330", BT="2330")
                if s_str in bt_data_names:
                    final_matched_symbols.add(bt_data_names[s_str])
                    
                # 情況 B: Picker 有 .TW，但 BT 沒有 (例如 Picker="2330.TW", BT="2330")
                elif s_str.replace('.TW', '') in bt_data_names:
                    final_matched_symbols.add(bt_data_names[s_str.replace('.TW', '')])
                
                # 情況 C: Picker 沒有 .TW，但 BT 有 (例如 Picker="2330", BT="2330.TW")
                elif f"{s_str}.TW" in bt_data_names:
                    final_matched_symbols.add(bt_data_names[f"{s_str}.TW"])

            # 更新策略的選股清單
            self.selected_symbols = final_matched_symbols
            self.last_selection_date = current_date
            
            # =========================================================
            # 5. 詳細日誌 (讓你確認是否對齊成功)
            # =========================================================
            if len(self.selected_symbols) > 0:
                # 為了避免 Log 太長，只印前 5 檔
                example_stocks = list(self.selected_symbols)[:5]
                self.log(
                    f'選股結果: Picker選出 {len(raw_picker_symbols)} 檔 -> '
                    f'與BT匹配成功 {len(self.selected_symbols)} 檔 '
                    f'(範例: {example_stocks} ...)'
                )
            else:
                # 如果 Picker 有選出東西，但匹配後是 0，代表代碼格式有問題
                if len(raw_picker_symbols) > 0:
                    self.log(
                        f'警告: 選股匹配失敗！Picker 選出 {len(raw_picker_symbols)} 檔 (如 {list(raw_picker_symbols)[0]}), '
                        f'但無法對應到 Backtrader 的資料名稱 (如 {self.datas[0]._name})'
                    )
                else:
                    self.log('選股完成: 無符合條件股票')

        except Exception as e:
            self.log(f'選股過程發生例外錯誤: {e}')
            import traceback
            traceback.print_exc() # 印出詳細錯誤以便除錯
            
            # 如果出錯，為了安全起見，可以選擇清空或保持原狀
            # self.selected_symbols = set()

    def log(self, txt):
        dt = self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} - {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 仍在撮合中
            return

        # 找出該訂單對應的股票
        symbol = None
        for s, o in self.orders.items():
            if o == order:
                symbol = s
                break
        
        symbol_str = f'[{symbol}] ' if symbol else ''

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'{symbol_str}BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Size: {order.executed.size}, Cost: {order.executed.value:.2f}'
                )
            elif order.issell():
                self.log(
                    f'{symbol_str}SELL EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Size: {order.executed.size}, Cost: {order.executed.value:.2f}'
                )

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'{symbol_str}Order Canceled/Margin/Rejected')

        # 無論如何，清掉掛單狀態
        if symbol:
            self.orders[symbol] = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        # 找出該交易對應的股票
        symbol = None
        for s in self.indicators.keys():
            if hasattr(trade.data, '_name') and trade.data._name == s:
                symbol = s
                break
        
        symbol_str = f'[{symbol}] ' if symbol else ''
        self.log(
            f'{symbol_str}TRADE PROFIT, Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}'
        )

    def next(self):
        # 獲取當前日期
        current_date = self.datas[0].datetime.date(0)
        self.trading_day_count += 1
        
        # 檢查是否需要執行選股（每個交易日或根據頻率）
        if (self.last_selection_date is None or 
            self.trading_day_count % self.p.rebalance_frequency == 0):
            self._perform_stock_selection(current_date)
        
        # 遍歷所有資料源（股票）
        for i, data in enumerate(self.datas):
            symbol = data._name if hasattr(data, '_name') else f'data{i}'
            
            # 檢查是否有掛單
            if self.orders[symbol] is not None:
                continue
            
            close = data.close[0]
            ind = self.indicators[symbol]
            N = ind['atr'][0]
            
            # [新增] 檢查指標是否準備好
            # 如果 ATR 還是 NaN，代表這支股票剛上市或資料剛開始，不能交易
            if math.isnan(ind['atr'][0]) or math.isnan(ind['s1_high'][0]):
                continue
            
            # 獲取當前持倉
            position = self.getposition(data)
            
            # 如果該股票不在選中列表中
            if symbol not in self.selected_symbols:
                # 如果有持倉，繼續執行出場邏輯（停損、技術指標出場）
                # 但不再建立新倉位
                if position:
                    # 執行出場邏輯
                    exit_s1 = close < ind['s1_low_exit'][0]
                    exit_s2 = close < ind['s2_low_exit'][0]
                    exit_stop = self.stop_prices[symbol] is not None and close < self.stop_prices[symbol]
                    
                    if exit_stop or exit_s1 or exit_s2:
                        self.log(
                            f'[{symbol}] EXIT LONG (被踢出選股池), Price: {close:.2f}, '
                            f'Stop: {self.stop_prices[symbol] if self.stop_prices[symbol] is not None else "nan"}, '
                            f'S1Low: {ind["s1_low_exit"][0]:.2f}, '
                            f'S2Low: {ind["s2_low_exit"][0]:.2f}'
                        )
                        self.orders[symbol] = self.close(data=data)
                        self.units[symbol] = 0
                        self.entry_prices[symbol] = None
                        self.stop_prices[symbol] = None
                        self.next_add_prices[symbol] = None
                # 如果沒有持倉，直接跳過（不建立新倉位）
                continue
            
            # 1) 沒有持倉 → 找入場機會（只有在選中列表中才會執行）
            if not position:
                self.units[symbol] = 0
                self.entry_prices[symbol] = None
                self.stop_prices[symbol] = None
                self.next_add_prices[symbol] = None

                # 只要任一系統出現「向上突破」訊號就建立多單
                long_signal_s1 = close > ind['s1_high'][0]
                long_signal_s2 = close > ind['s2_high'][0]

                # 檢查 N 是否有效（不是 None、NaN 且 > 0）
                if (long_signal_s1 or long_signal_s2) and N is not None and not pd.isna(N) and N > 0:
                    size = self._calc_unit_size(symbol)
                    if size <= 0:
                        continue  # 資金或 N 不適合開倉

                    # 確保 size 是整數（股數不可分割）
                    size = int(size)
                    
                    # 建立第一個 unit
                    self.orders[symbol] = self.buy(data=data, size=size)
                    self.units[symbol] = 1
                    self.entry_prices[symbol] = close

                    # 初始停損價：入場價 - 2N
                    self.stop_prices[symbol] = self.entry_prices[symbol] - self.p.atr_mult_stop * N

                    # 下一次加碼價：入場價 + 0.5N
                    self.next_add_prices[symbol] = self.entry_prices[symbol] + self.p.atr_step_pyramid * N

                    self.log(
                        f'[{symbol}] ENTRY LONG, Price: {close:.2f}, Size: {size}, '
                        f'N: {N:.2f}, Stop: {self.stop_prices[symbol]:.2f}, '
                        f'Next Add: {self.next_add_prices[symbol]:.2f}'
                    )

            # 2) 已經有多頭部位 → 管理出場 & 加碼
            else:
                # 出場條件：
                # a) 跌破停損價
                # b) 系統1：跌破 10 日低點
                # c) 系統2：跌破 20 日低點
                exit_s1 = close < ind['s1_low_exit'][0]
                exit_s2 = close < ind['s2_low_exit'][0]
                exit_stop = self.stop_prices[symbol] is not None and close < self.stop_prices[symbol]

                if exit_stop or exit_s1 or exit_s2:
                    self.log(
                        f'[{symbol}] EXIT LONG, Price: {close:.2f}, '
                        f'Stop: {self.stop_prices[symbol] if self.stop_prices[symbol] is not None else "nan"}, '
                        f'S1Low: {ind["s1_low_exit"][0]:.2f}, '
                        f'S2Low: {ind["s2_low_exit"][0]:.2f}'
                    )
                    self.orders[symbol] = self.close(data=data)
                    self.units[symbol] = 0
                    self.entry_prices[symbol] = None
                    self.stop_prices[symbol] = None
                    self.next_add_prices[symbol] = None
                    continue  # 出場後本 bar 不再加碼

                # 加碼邏輯：只在順勢 & 未達 max_units 時加碼
                if (
                    self.units[symbol] > 0
                    and self.units[symbol] < self.p.max_units
                    and self.next_add_prices[symbol] is not None
                    and N is not None
                    and not pd.isna(N)
                    and N > 0
                ):
                    # 價格已經走到加碼價以上 → 再買一個 unit
                    if close >= self.next_add_prices[symbol]:
                        size = self._calc_unit_size(symbol)
                        if size > 0:
                            # 確保 size 是整數（股數不可分割）
                            size = int(size)
                            self.orders[symbol] = self.buy(data=data, size=size)
                            self.units[symbol] += 1

                            # 設定下一階加碼價：在上一階基礎上再 +0.5N
                            self.next_add_prices[symbol] = self.next_add_prices[symbol] + self.p.atr_step_pyramid * N

                            self.log(
                                f'[{symbol}] PYRAMID ADD, Price: {close:.2f}, Size: {size}, '
                                f'Units: {self.units[symbol]}, Next Add: {self.next_add_prices[symbol]:.2f}'
                            )

In [2]:
import backtrader as bt
from datetime import datetime, timedelta

# ===== 1. 設定回測參數 =====
# 回測期間：2024-01-01 到 2024-12-31
BACKTEST_START = '2024-01-01'
BACKTEST_END = '2024-12-31'

# 選股需要一個月前的資料，所以資料下載期間要往前推一個月
DATA_START = (datetime.strptime(BACKTEST_START, '%Y-%m-%d') - timedelta(days=30)).strftime('%Y-%m-%d')
DATA_END = BACKTEST_END

print("=" * 60)
print("步驟 1: 下載台灣所有股票資料（包含選股所需的歷史資料）")
print("=" * 60)

# ===== 2. 下載台灣所有股票資料 =====
# 下載從一個月前到回測結束的所有資料（用於選股）
full_data_manager = StockDataManager(
    country_code='TW',
    start_date=DATA_START,
    end_date=DATA_END
)

步驟 1: 下載台灣所有股票資料（包含選股所需的歷史資料）
正在從台灣證交所 API 抓取股票代號...
成功抓取 1321 檔台灣股票代號
正在使用 yfinance 下載 1321 檔股票資料...
正在下載第 1 批（共 100 檔）...
正在下載第 2 批（共 100 檔）...



21 Failed downloads:
['00981T.TW', '009803.TW', '009800.TW', '009808.TW', '009805.TW', '009804.TW', '00980A.TW', '00981A.TW', '00982D.TW', '00983D.TW', '009810.TW', '00984A.TW', '009813.TW', '009812.TW', '009809.TW', '00985A.TW', '00983A.TW', '009801.TW', '009811.TW', '009802.TW', '00982A.TW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-02 -> 2024-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1701446400, endDate = 1735574400")')


正在下載第 3 批（共 100 檔）...



20 Failed downloads:
['020031.TW', '020012.TW', '02001R.TW', '020030.TW', '020011.TW', '020034.TW', '020000.TW', '020037.TW', '020020.TW', '02001L.TW', '020029.TW', '020036.TW', '020039.TW', '02001S.TW', '020038.TW', '020028.TW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-02 -> 2024-12-31)')
['00988A.TW', '00989A.TW', '00985B.TW', '00986A.TW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-02 -> 2024-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1701446400, endDate = 1735574400")')


正在下載第 4 批（共 100 檔）...
正在下載第 5 批（共 100 檔）...


...

正在下載第 11 批（共 100 檔）...
正在下載第 12 批（共 100 檔）...



8 Failed downloads:
['6918.TW', '6589.TW', '6965.TW', '6887.TW', '6909.TW', '7610.TW', '6944.TW', '6936.TW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-02 -> 2024-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1701446400, endDate = 1735574400")')


正在下載第 13 批（共 100 檔）...



7 Failed downloads:
['7740.TW', '7736.TW', '7788.TW', '7780.TW', '7786.TW', '7750.TW', '7749.TW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-02 -> 2024-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1701446400, endDate = 1735574400")')


正在下載第 14 批（共 21 檔）...
------------------------------
已成功下載 1321 檔股票資料
  - 0050.TW: 261 筆資料，日期範圍 2023-12-04 至 2024-12-30
  - 0051.TW: 261 筆資料，日期範圍 2023-12-04 至 2024-12-30
  - 0053.TW: 261 筆資料，日期範圍 2023-12-04 至 2024-12-30
  - 0055.TW: 261 筆資料，日期範圍 2023-12-04 至 2024-12-30
  - 0056.TW: 261 筆資料，日期範圍 2023-12-04 至 2024-12-30
  ... 還有 1316 檔股票
------------------------------


In [12]:
print("\n" + "=" * 60)
print("步驟 2: 切出回測期間的資料給 backtrader")
print("=" * 60)

# ===== 3. 切出回測期間的資料 =====
# 從完整資料中切出回測期間的資料
backtest_data_manager = full_data_manager.subset(BACKTEST_START, BACKTEST_END)

print(f"\n回測期間資料: {len(backtest_data_manager.stock_data)} 檔股票")

# ===== 4. 設定 backtrader =====
cerebro = bt.Cerebro()

added_count = 0
for symbol, df in backtest_data_manager.stock_data.items():
    if df.empty:
        continue
    try:
        # 1. 確保索引格式
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
        
        # [新增] 2. 資料清洗：非常重要！
        # 填補空缺值 (Forward Fill)，如果還有 NaN 則刪除該行
        df = df.ffill().dropna()

        # [新增] 3. 檢查資料長度
        # 如果資料少於策略需要的最小天數 (例如 55 天)，直接跳過，不然指標算不出來會報錯
        if len(df) < 60: 
            print(f"跳過 {symbol}: 資料長度不足 ({len(df)} days)")
            continue

        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        if all(col in df.columns for col in required_cols):
            datafeed = bt.feeds.PandasData(
                dataname=df,
                datetime=None,
                open=0, high=1, low=2, close=3, volume=4,
                openinterest=-1
            )
            datafeed._name = symbol
            cerebro.adddata(datafeed)
            added_count += 1
            
    except Exception as e:
        print(f"警告: 無法添加 {symbol} 的資料: {e}")
        continue

print(f"\n成功添加 {added_count} 檔股票的資料到 backtrader")

# ===== 5. 添加策略 =====
cerebro.addstrategy(
    TurtleLongWithSelection,
    selection_data_manager=full_data_manager,  # 傳入完整資料管理器（用於選股）
    selection_lookback_days=30,  # 選股回測時間：30天
    rebalance_frequency=1,  # 每個交易日都重新選股
)

# ===== 6. 設定經紀商參數 =====
cerebro.broker.setcash(1000000.0)  # 初始資金 100 萬
cerebro.broker.setcommission(commission=0.001)  # 手續費 0.1%

# ===== 7. 執行回測 =====
print("\n" + "=" * 60)
print("步驟 3: 執行回測")
print("=" * 60)
print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')

cerebro.run()

print(f'\nFinal Portfolio Value: {cerebro.broker.getvalue():.2f}')
print(f'Total Return: {(cerebro.broker.getvalue() / 1000000.0 - 1) * 100:.2f}%')


步驟 2: 切出回測期間的資料給 backtrader

回測期間資料: 1321 檔股票
跳過 00938.TW: 資料長度不足 (55 days)
跳過 00961.TW: 資料長度不足 (59 days)
跳過 00962.TW: 資料長度不足 (24 days)
跳過 00963.TW: 資料長度不足 (33 days)
跳過 00964.TW: 資料長度不足 (33 days)
跳過 00965.TW: 資料長度不足 (36 days)
跳過 00971.TW: 資料長度不足 (2 days)
跳過 00972.TW: 資料長度不足 (1 days)
跳過 009800.TW: 資料長度不足 (0 days)
跳過 009801.TW: 資料長度不足 (0 days)
跳過 009802.TW: 資料長度不足 (0 days)
跳過 009803.TW: 資料長度不足 (0 days)
跳過 009804.TW: 資料長度不足 (0 days)
跳過 009805.TW: 資料長度不足 (0 days)
跳過 009808.TW: 資料長度不足 (0 days)
跳過 009809.TW: 資料長度不足 (0 days)
跳過 00980A.TW: 資料長度不足 (0 days)
跳過 009810.TW: 資料長度不足 (0 days)
跳過 009811.TW: 資料長度不足 (0 days)
跳過 009812.TW: 資料長度不足 (0 days)
跳過 009813.TW: 資料長度不足 (0 days)
跳過 00981A.TW: 資料長度不足 (0 days)
跳過 00981T.TW: 資料長度不足 (0 days)
跳過 00982A.TW: 資料長度不足 (0 days)
跳過 00982D.TW: 資料長度不足 (0 days)
跳過 00983A.TW: 資料長度不足 (0 days)
跳過 00983D.TW: 資料長度不足 (0 days)
跳過 00984A.TW: 資料長度不足 (0 days)
跳過 00985A.TW: 資料長度不足 (0 days)
跳過 00985B.TW: 資料長度不足 (0 days)
跳過 00986A.TW: 資料長度不足 (0 days)
跳過 00988A.TW: 資料長度不足 (0 d